In [1]:
import  pandas  as  pd
import numpy  as np
train=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/numerai_training_data.csv')
print(train.head(3))
train1=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/trainCat.csv')
train['sums']=train1['sums']
train['kinetic']=train1['kinetic']
y=train['target']
train=train.drop(['target'],axis=1)
print(train.shape)
print(y.shape)
test=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/numerai_tournament_data.csv')
test1=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/testCat.csv')
test['sums']=test1['sums']
test['kinetic']=test1['kinetic']

id=test['t_id']
test=test.drop(['t_id'],axis=1)
sub=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/example_predictions.csv')
trainvec=np.array(train)
testvec=np.array(test)
y=np.array(y)

   feature1  feature2  feature3  feature4  feature5  feature6  feature7  \
0  0.185868  0.895807  0.596433  0.028381  0.053369  0.202876  0.912054   
1  0.709165  0.967550  0.529121  0.611704  0.961349  0.786213  0.489511   
2  0.526553  0.015789  0.940873  0.867239  0.084545  0.033899  0.412348   

   feature8  feature9  feature10   ...    feature13  feature14  feature15  \
0  0.669988  0.143481   0.132973   ...     0.855909   0.842887   0.913296   
1  0.607460  0.715500   0.476030   ...     0.517913   0.279985   0.938501   
2  0.538480  0.122290   0.370455   ...     0.992078   0.878373   0.090039   

   feature16  feature17  feature18  feature19  feature20  feature21  target  
0   0.938493   0.393211   0.211985   0.337367   0.898316   0.245452       0  
1   0.490024   0.630741   0.117521   0.917893   0.894724   0.897643       1  
2   0.379432   0.271485   0.361007   0.379741   0.028594   0.351007       0  

[3 rows x 22 columns]
(136573, 23)
(136573,)


In [2]:
train.head(3)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,sums,kinetic
0,0.185868,0.895807,0.596433,0.028381,0.053369,0.202876,0.912054,0.669988,0.143481,0.132973,...,0.842887,0.913296,0.938493,0.393211,0.211985,0.337367,0.898316,0.245452,6.974981,0.197279
1,0.709165,0.967550,0.529121,0.611704,0.961349,0.786213,0.489511,0.607460,0.715500,0.476030,...,0.279985,0.938501,0.490024,0.630741,0.117521,0.917893,0.894724,0.897643,10.475775,0.156463
2,0.526553,0.015789,0.940873,0.867239,0.084545,0.033899,0.412348,0.538480,0.122290,0.370455,...,0.878373,0.090039,0.379432,0.271485,0.361007,0.379741,0.028594,0.351007,5.252490,0.174603


In [3]:
#shuffle  data,  scale features  between  [0,1] and split  for  ground  truth  optimization
from sklearn import preprocessing
trainvec=np.array(train)
 
from sklearn.cross_validation import train_test_split
train, eval1, ytrain, yeval = train_test_split(trainvec, y, test_size=0.3)

In [4]:
#lets compute  theyr score  now
 
from sklearn.metrics import log_loss
class generation(object):
    def __init__(self,p):
        """here  the  class is initialized with initial(stone age population)"""
        self.gen=p
    def fitness(self):
        """here   for the  given population I compute the  fitness for each individual
        meaning  result of  competition metric(log loss) and   some eval  yeval.
        Neither  the  less  every function  will  return   population dict"""
        for  dude in range(len(self.gen['individuals'])):
            avg=np.average(train[:,self.gen['individuals'][dude]],axis=1)
            avg_val=np.average(eval1[:,self.gen['individuals'][dude]],axis=1)
            self.gen['fitness_val'][dude]=log_loss(yeval,avg_val)
            
 
            self.gen['fitness'][dude]=log_loss(ytrain,avg)
    
        return self.gen
    def population_statistics(self):
        """Will bring  min , max  and median  of population fitness. I choosed  median instead of  mean
        because  it is   not  sensible to ouliers.Will also return dict"""
        self.gen['median_score']=np.median(self.gen['fitness'])
        self.gen['min_score']=np.min(self.gen['fitness'])
        self.gen['max_score']=np.max(self.gen['fitness'])
        self.gen['min_val']=np.min(self.gen['fitness_val'])
        self.gen['median_val']=np.median(self.gen['fitness_val'])
        return self.gen
        
        
        
 
        
        
            

In [41]:
#if the  corss - over  will not be correct  will see   no evolution  :((  and  will be just heuristic search
import heapq
class  evolving_things(object):
    def __init__(self,current_gen):
        
        self.current_generation=current_gen
        
    def eugenic_dudes(self,percentage):
        """gets top  percent  performmer  individuals . """
        size=int(percentage*len(self.current_generation['individuals']))
        performers_idx=heapq.nsmallest(size, range(len(self.current_generation['fitness'])),
                                       self.current_generation['fitness'].__getitem__)
        #randomly  add other individuals  to  promote  genetic diversity
        #http://lethain.com/genetic-algorithms-cool-name-damn-simple/
        #there  they are choosen  by worse  guys  we will  choose  from entire population
        #for some_dude in range(len(self.current_generation['individuals'])):
        #    if random.random()<0.1:
                 
        #        performers_idx.append(some_dude)
        return performers_idx
    
    def mutation(self,percentage):
        """every individual   will have 0.1 probability  of mutation"""
        """The mutation   is made in the next manner:
        Will peek some random individual  from top 20 best performing on  train  val and some from 
        top 20 best performming  on  holdout data set and  glue them togheter
        """
        percentage=0.3
        size=int(percentage*len(self.current_generation['individuals']))
        performers_train=heapq.nsmallest(size, range(len(self.current_generation['fitness'])),
                                       self.current_generation['fitness'].__getitem__)
        performers_val=heapq.nsmallest(size, range(len(self.current_generation['fitness_val'])),
                                       self.current_generation['fitness'].__getitem__)
        for mutant in range(len(self.current_generation['individuals'])):
            if random.random()<0.007:
                cromo1=np.random.choice(performers_train)
                cromo2=np.random.choice(performers_val)
                mutant_dude=self.current_generation['individuals'][cromo1]+self.current_generation['individuals'][cromo2]
                self.current_generation['individuals'][mutant]=mutant_dude
             
                
        
        
    
    def make_newpop(self,percentage):
        newgen={'max_score':0,'min_score':0,'median_score':0,'individuals':[[] for individual in  
                                                                                 range(80)]}
        newgen['fitness']=[[] for fitness in range(len(newgen['individuals']))]
        newgen['fitness_val']=[[] for fitness in range(len(newgen['individuals']))]
        #the   new population  is   initial  filled  with  top performmers  and other  empty  lists
        #that need to  be filled in with  offsprings
        
        idx=self.eugenic_dudes(percentage)
        for performer in  idx:
            newgen['individuals'][performer]=self.current_generation['individuals'][performer]

    
        return(newgen)
    
        
    def  cross_over(self,percentage):
        """do cross over will cross  over  individuals   in  order to   fill in new  population"""
        self.mutation(0.2)
 
        newpop=self.make_newpop(percentage)
        idx=self.eugenic_dudes(percentage)
        #for loop  positions   that  are   empty - there a new individual will be  born 
        for dude in range(len(newpop['individuals'])):
            if len(newpop['individuals'][dude])==0:
                 #from the  list  of indexes of  top  performmers  select    mother  and  father
                    parents=np.random.choice(idx,2,replace=False)
                 
                    mother=parents[0]
                    father=parents[1]
                    if random.random()<0.3:
                        kid=newpop['individuals'][mother]+newpop['individuals'][father]
                        newpop['individuals'][dude]=list(set(kid))
                        
    
                        
                    else:
                        cromo1=len(newpop['individuals'][mother])
                        cromo2=len(newpop['individuals'][father])
                        inheritance_mother=newpop['individuals'][mother][:int(cromo1/2)]
                        inheritance_father=newpop['individuals'][father][int(cromo2/2):]
                        kid=inheritance_mother+inheritance_father
                        newpop['individuals'][dude]=list(set(kid))
                        
                    
                     
                        
                         
 
        
                    
                    
        return(newpop)#return  a dict  with  generation
                

In [42]:
#do  recursion 
pop={'max_score':0,'min_score':0,'median_score':0,'individuals':[[] for individual in  
                                                                 range(80)],
    'median_val':0,'min_val':0}
pop['fitness']=[[] for fitness in range(len(pop['individuals']))]
pop['fitness_val']=[[] for fitness in range(len(pop['individuals']))]
for i in range(80):
    individual=list(np.random.choice(21,13))
    pop['individuals'][i]=individual


In [43]:
import random
trace=[]
for time_is_flying   in range(20):
    #start with the orininal poulation:
    print("Evolving generation number :", time_is_flying)

    #grade  fred and barny
    g=generation(pop)#makes  class with pop
    g=g.fitness()#sets fitness
    g=generation(g)
    g=g.population_statistics()#brings a  dict
    trace.append(g)
 
    print("median :",g['median_score']," min :",g['min_score'],"med val",g['median_val'],"min val",g['min_val'])
    e=evolving_things(g)
    e.eugenic_dudes(0.4)
    e.make_newpop(0.4)
    pop=e.cross_over(0.4)


 
    
    
    
    
    
    
 

Evolving generation number : 0
median : 0.720005480759  min : 0.705087808993 med val 0.718685581146 min val 0.70452016954
Evolving generation number : 1
median : 0.712337021009  min : 0.705087808993 med val 0.711328679377 min val 0.70452016954
Evolving generation number : 2
median : 0.707468164611  min : 0.704426244538 med val 0.706807856852 min val 0.703741625554
Evolving generation number : 3
median : 0.705944113633  min : 0.703736429381 med val 0.705171280206 min val 0.702915623977
Evolving generation number : 4
median : 0.70491285222  min : 0.703642701578 med val 0.70406183651 min val 0.702669848577
Evolving generation number : 5
median : 0.704523630242  min : 0.703450270751 med val 0.703710369608 min val 0.70252556454
Evolving generation number : 6
median : 0.703973284858  min : 0.703404880171 med val 0.70307479735 min val 0.70252556454
Evolving generation number : 7
median : 0.703768459358  min : 0.703404880171 med val 0.702915623977 min val 0.70252556454
Evolving generation numb

In [71]:
train=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/numerai_training_data.csv')
test=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/numerai_tournament_data.csv')
originaltrain=train
originaltest=test

In [72]:
feat=trace[3]#pontentialy  choose acorgin to evolution errors- the  last  ones will not  be good
             #try first with  small values  to see if    does  have  mistakes
              #from those  3000 will remain  fewer cause they  are  highly correlated
trainvec=np.array(train)
test=test.drop(['t_id'],axis=1)
testvec=np.array(testvec)
for   i in  range(79):
    f=feat['individuals'][i]
    newfeature=np.average(trainvec[:,f],axis=1)
    originaltrain['feat'+str(i)]=newfeature
    newfeature=np.average(testvec[:,f],axis=1)
    originaltest['feat'+str(i)]=newfeature
    
 


In [ ]:
originaltrain.to_csv('/media/machine_learning/A80C461E0C45E7C01/all/trainGA.csv',index=False)
originaltest.to_csv('/media/machine_learning/A80C461E0C45E7C01/all/testGA.csv',index=False)